In [1]:
import os
from snowflake.snowpark import Session
import pandas as pd
import requests

In [2]:
#Connecting to snowflake
connection_parameters = {
"account": "ld77469.uae-north.azure",
"user": "GOKS98",
"password": "March20311",
"role": "ACCOUNTADMIN",
"warehouse": "SNOWFLAKE_WAREHOUSE",
"database": "OIL_AND_GAS",
"schema": "CONFORMED"
 }

session = Session.builder.configs(connection_parameters).create()

In [3]:
#Creating a Dataframe
df_table = session.table("SNOWFLAKE_SAMPLE_DATA.TPCDS_SF10TCL.WAREHOUSE")
df_table.show()

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"W_WAREHOUSE_SK"  |"W_WAREHOUSE_ID"  |"W_WAREHOUSE_NAME"    |"W_WAREHOUSE_SQ_FT"  |"W_STREET_NUMBER"  |"W_STREET_NAME"  |"W_STREET_TYPE"  |"W_SUITE_NUMBER"  |"W_CITY"       |"W_COUNTY"         |"W_STATE"  |"W_ZIP"  |"W_COUNTRY"    |"W_GMT_OFFSET"  |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|1                 |AAAAAAAABAAAAAAA  |Conventional childr   |977787               |651                |6th              |Parkway          |Suite 470         |Shiloh         |Franklin Parish    |LA         |79275    |United States  |-6.00         

In [4]:
df_table_sql = session.sql("SELECT COUNT(*) FROM SNOWFLAKE_SAMPLE_DATA.TPCDS_SF10TCL.WAREHOUSE")
df_table_sql.show()

--------------
|"COUNT(*)"  |
--------------
|25          |
--------------



In [5]:
#Checking the columns
df_table_sql = session.sql("""SELECT W_STATE, COUNT(W_WAREHOUSE_NAME) FROM SNOWFLAKE_SAMPLE_DATA.TPCDS_SF10TCL.WAREHOUSE
                              GROUP BY W_STATE
                              ORDER BY 2 DESC""")
df_table_sql.show()


-----------------------------------------
|"W_STATE"  |"COUNT(W_WAREHOUSE_NAME)"  |
-----------------------------------------
|CO         |4                          |
|WV         |4                          |
|LA         |2                          |
|SD         |2                          |
|NC         |2                          |
|MO         |2                          |
|NE         |1                          |
|MI         |1                          |
|TN         |1                          |
|NM         |1                          |
-----------------------------------------



In [6]:
#Getting weather data from API for every zipcode
df_pd = df_table.to_pandas()
df_pd.head()

,W_WAREHOUSE_SK,W_WAREHOUSE_ID,W_WAREHOUSE_NAME,W_WAREHOUSE_SQ_FT,W_STREET_NUMBER,W_STREET_NAME,W_STREET_TYPE,W_SUITE_NUMBER,W_CITY,W_COUNTY,W_STATE,W_ZIP,W_COUNTRY,W_GMT_OFFSET
0,1,AAAAAAAABAAAAAAA,Conventional childr,977787.0,651,6th,Parkway,Suite 470,Shiloh,Franklin Parish,LA,79275,United States,-6.0
1,2,AAAAAAAACAAAAAAA,Important issues liv,138504.0,600,View First,Avenue,Suite P,Greenwood,Ziebach County,SD,58828,United States,-6.0
2,3,AAAAAAAADAAAAAAA,Doors canno,294242.0,534,Ash Laurel,Dr.,Suite 0,Cedar Grove,Raleigh County,WV,20411,United States,-5.0
3,4,AAAAAAAAEAAAAAAA,Bad cards must make.,621234.0,368,Wilson Elm,Drive,Suite 80,Oakland,Gage County,NE,69843,United States,-6.0
4,5,AAAAAAAAFAAAAAAA,None,NaN,None,None,None,None,Midway,Mesa County,CO,81904,United States,NaN


In [7]:
#Getting weather data from API for every zipcode
json_list = []
for i in df_pd['W_ZIP']:
    response = requests.get(f"http://api.weatherapi.com/v1/current.json?key=c75b271f50ea4479a7361323220211&q={i}")
    data = response.json()
    json_list.append(data)
json_list

[{'error': {'code': 1006, 'message': 'No matching location found.'}},
 {'error': {'code': 1006, 'message': 'No matching location found.'}},
 {'location': {'name': 'Washington',
   'region': 'District of Columbia',
   'country': 'USA',
   'lat': 38.89,
   'lon': -77.02,
   'tz_id': 'America/New_York',
   'localtime_epoch': 1667372239,
   'localtime': '2022-11-02 2:57'},
  'current': {'last_updated_epoch': 1667371500,
   'last_updated': '2022-11-02 02:45',
   'temp_c': 15.0,
   'temp_f': 59.0,
   'is_day': 0,
   'condition': {'text': 'Clear',
    'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png',
    'code': 1000},
   'wind_mph': 2.2,
   'wind_kph': 3.6,
   'wind_degree': 322,
   'wind_dir': 'NW',
   'pressure_mb': 1023.0,
   'pressure_in': 30.21,
   'precip_mm': 0.0,
   'precip_in': 0.0,
   'humidity': 81,
   'cloud': 0,
   'feelslike_c': 14.5,
   'feelslike_f': 58.0,
   'vis_km': 16.0,
   'vis_miles': 9.0,
   'uv': 1.0,
   'gust_mph': 12.1,
   'gust_kph': 19.4}},
 {'error': {'

In [8]:
zip_code = df_pd['W_ZIP']
data = list(zip(zip_code, json_list))

weather_df = pd.DataFrame(data=data, columns=['W_ZIP', 'WEATHER_JSON'])
weather_df

,W_ZIP,WEATHER_JSON
0,79275,"{'error': {'code': 1006, 'message': 'No matchi..."
1,58828,"{'error': {'code': 1006, 'message': 'No matchi..."
2,20411,"{'location': {'name': 'Washington', 'region': ..."
3,69843,"{'error': {'code': 1006, 'message': 'No matchi..."
4,81904,"{'error': {'code': 1006, 'message': 'No matchi..."
5,20411,"{'location': {'name': 'Washington', 'region': ..."
6,24593,"{'location': {'name': 'Spout Spring', 'region'..."
7,43604,"{'location': {'name': 'Toledo', 'region': 'Ohi..."
8,37057,"{'location': {'name': 'Dixon Springs', 'region..."
9,21904,"{'location': {'name': 'Port Deposit', 'region'..."


In [9]:
#Joining tables
snowflake_df = df_pd.merge(weather_df, how='inner', on='W_ZIP')
snowflake_df.head()

,W_WAREHOUSE_SK,W_WAREHOUSE_ID,W_WAREHOUSE_NAME,W_WAREHOUSE_SQ_FT,W_STREET_NUMBER,W_STREET_NAME,W_STREET_TYPE,W_SUITE_NUMBER,W_CITY,W_COUNTY,W_STATE,W_ZIP,W_COUNTRY,W_GMT_OFFSET,WEATHER_JSON
0,1,AAAAAAAABAAAAAAA,Conventional childr,977787.0,651,6th,Parkway,Suite 470,Shiloh,Franklin Parish,LA,79275,United States,-6.0,"{'error': {'code': 1006, 'message': 'No matchi..."
1,2,AAAAAAAACAAAAAAA,Important issues liv,138504.0,600,View First,Avenue,Suite P,Greenwood,Ziebach County,SD,58828,United States,-6.0,"{'error': {'code': 1006, 'message': 'No matchi..."
2,3,AAAAAAAADAAAAAAA,Doors canno,294242.0,534,Ash Laurel,Dr.,Suite 0,Cedar Grove,Raleigh County,WV,20411,United States,-5.0,"{'location': {'name': 'Washington', 'region': ..."
3,3,AAAAAAAADAAAAAAA,Doors canno,294242.0,534,Ash Laurel,Dr.,Suite 0,Cedar Grove,Raleigh County,WV,20411,United States,-5.0,"{'location': {'name': 'Washington', 'region': ..."
4,6,AAAAAAAAGAAAAAAA,"Local, mass universi",838797.0,957,Lincoln Adams,Dr.,Suite X,Cedar Grove,Raleigh County,WV,20411,United States,-5.0,"{'location': {'name': 'Washington', 'region': ..."


In [10]:
snowflake_df_1 = session.create_dataframe(snowflake_df)
snowflake_df_1.show()

create_temp_table is deprecated. We still respect this parameter when it is True but please consider using `table_type="temporary"` instead.


-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"W_WAREHOUSE_SK"  |"W_WAREHOUSE_ID"  |"W_WAREHOUSE_NAME"    |"W_WAREHOUSE_SQ_FT"  |"W_STREET_NUMBER"  |"W_STREET_NAME"  |"W_STREET_TYPE"  |"W_SUITE_NUMBER"  |"W_CITY"       |"W_COUNTY"       |"W_STATE"  |"W_ZIP"  |"W_COUNTRY"    |"W_GMT_OFFSET"  |"WEATHER_JSON"                                      |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|1                 |AAAAAAAABAAAAAAA  |Conventional childr   |977787.0             |651       

In [11]:
snowflake_df_1.write.mode("overwrite").save_as_table("w_warehouse_weather")